# ICT for Development - Collaborative API library

## Overview

This notebook aims to compile beginner-friendly documentation of APis relevant to the work of civic actors active in the field of international development. The project was started in 2024 by [Cedric Lombion](https://civicliteraci.es) to support his teaching at the University of Turin's ICT for Development and Social Good master's programme. Nonetheless, the project bears no formal affiliation with the university. 

Each entry aims to document key information and examples about the API being documented, with a focus on making them more accessible to beginners to APIs and command line scripting.

## Contributing

1. Fork the repository
2. add your API descriptioni and examples
3. Ensure that your examples are well documented an suitable for beginners
4. send a pull request which will be reviewed for addition. 

For suggestion or improvements, open an issue on the repository. Alternatively, contact us at contact [at] civicliteraci.es

## Genderize API
* **main website**: https://genderize.io
* **API docs**: https://genderize.io/documentation
* **Purpose:** predicts the gender of an individual based on their name and (optionally) country
* **Limitations:** 
    * binary gender model
    * up to 100 names/day
    * up to 10 names per request (batch request)
    * using localization on a batch request will apply the given country_id to every name in the batch


### Basic example

In [8]:
# We use the option --data-urlencode to make it easy to include names with special characters
curl -G "https://api.genderize.io/" --data-urlencode "name=Marie Anne" --data-urlencode "country=FR"

{"count":481613,"name":"Marie Anne","gender":"female","probability":1.0}


### Multiple names in a request

A loop can be used to construct a request that includes multiple names drawn from an array.  
the API accepts only up to 10 names.  
Adding a localization will apply to all names in the request.

In [13]:
# Defining our arrays

name_list=("yamen" "lawson" "maria" "alejandro")
country=("sy" "tg" "it" "es")

# Start building the curl command
request="curl -s -G \"https://api.genderize.io?\""

# Add each name to the curl command with --data-urlencode
for name in "${name_list[@]}"
do
    request+=" --data-urlencode \"name[]=${name}\""
done

# Check that the command is properly formed
echo "$request"

# Excute the command
bash -c "$request" | jq

curl -s -G "https://api.genderize.io?" --data-urlencode "name[]=yamen" --data-urlencode "name[]=lawson" --data-urlencode "name[]=maria" --data-urlencode "name[]=alejandro"
[
  {
    "count": 814,
    "name": "lawson",
    "gender": "male",
    "probability": 0.88
  },
  {
    "count": 2687884,
    "name": "maria",
    "gender": "female",
    "probability": 1
  },
  {
    "count": 386853,
    "name": "alejandro",
    "gender": "male",
    "probability": 1
  }
]


### Automate name and country requests and CSV export

We can create a loop that makes it possible to automate requests with both the name and country included.  
We will process the returned json with jq to transform the results into tabular data.

In [14]:
# we declare our arrays
name_list=("yamen" "lawson" "maria" "alejandro")
country=("sy" "tg" "it" "es")

# we count and store the number of values in our array to reuse it in the loop
length=${#country[@]}

# we loop through the arrays
for ((i=0; i<length; i++))
do
    curl -s -G "https://api.genderize.io/" \
    --data-urlencode "name=${name_list[$i]}" \
    --data-urlencode "country_id=${country[$i]}" |
    jq '[.name, .gender] | @csv'
done

"\"yamen\",\"male\""
"\"lawson\",\"male\""
"\"maria\",\"female\""
"\"alejandro\",\"male\""


## SerpAPI
* **url**: https://serpapi.com/
* **Purpose**: getting JSON formatted google results
* **Constraints** 1000 free calls / day

### Simple query filtered with jq

Example of a basic query.  
The jq filter helps with producing a smaller output as the original JSON takes a lot of space.

In [8]:
# the jq code includes only the first five entries and identify their top level objects.

curl -s "https://serpapi.com/search.json?q=Coffee&location=Austin,+Texas,+United+States&hl=en&gl=us&google_domain=google.com" \
| jq 'to_entries[0:5] | .[] | {key: .key, type: (.value | type)}'


{
  "key": "search_metadata",
  "type": "object"
}
{
  "key": "search_parameters",
  "type": "object"
}
{
  "key": "search_information",
  "type": "object"
}
{
  "key": "local_map",
  "type": "object"
}
{
  "key": "local_results",
  "type": "object"
}


### Filtered data and export to CSV

In [9]:
#Same query, but with an explicity attribute filter and csv converstion with jq

curl -s "https://serpapi.com/search.json?q=Coffee&location=Austin,+Texas,+United+States&hl=en&gl=us&google_domain=google.com" \
| jq '.organic_results[] | [.title,.link, .snippet] | @csv'

"\"Coffee\",\"https://en.wikipedia.org/wiki/Coffee\",\"Coffee is a beverage brewed from roasted coffee beans. Darkly colored, bitter, and slightly acidic, coffee has a stimulating effect on humans, primarily due ...\""
"\"What is Coffee?\",\"https://www.ncausa.org/About-Coffee/What-is-Coffee\",\"Coffee traces its origin to a genus of plants known as Coffea. Within the genus there are over 500 genera and 6,000 species of tropical trees and shrubs.\""
"\"Starbucks Coffee Company\",\"https://www.starbucks.com/\",\"More than just great coffee. Explore the menu, sign up for Starbucks® Rewards, manage your gift card and more.\""
"\"Coffee | Origin, Types, Uses, History, & Facts\",\"https://www.britannica.com/topic/coffee\",\"Coffee, beverage brewed from the roasted and ground seeds of the tropical evergreen coffee plant of African origin. Coffee is one of the ...\""
"\"Coffee | The Nutrition Source - hsph.harvard.edu\",\"https://www.hsph.harvard.edu/nutritionsource/food-features/coffee/\",\"

## Country Info API

* **main website**: https://countryinfoapi.com
* **url**: https://countryinfoapi.com
* **Purpose**: fetch additional info about a country/continent etc
* **limitations:** 



In [50]:
countries=("France" "Togo" "Italy" "Germany" "Syria")

for i in ${countries[@]}
do  
    curl -s "https://countryinfoapi.com/api/countries/name/$i" | jq '[.name,.region] | @csv'
done

"\"France\",\"Europe\""
"\"Togo\",\"Africa\""
"\"Italy\",\"Europe\""
"\"Germany\",\"Europe\""
"\"Syria\",\"Asia\""


In [37]:
i="Ivory Coast"
clean=$(echo "${i// /+}")
echo "$clean"

Ivory+Coast


In [39]:
for i in ${countries[@]}
do echo "$i"
done

France
Togo
Italy
Germany
Syria


## Open Meteo API

* **main website**: 
* **API docs**: 
* **Purpose:** 
* **Limitations:** 

## Healthsites.io API

* **main website**: 
* **API docs**: 
* **Purpose:** 
* **Limitations:** 

# World Bank API
by Alejandro Sainz-Pardo

* **main website**: [World Bank API](https://documents.worldbank.org/en/publication/documents-reports/api) 
* **API docs**: All details on how to work with the API in the [link](https://documents.worldbank.org/en/publication/documents-reports/api)
* **Purpose:** The World Bank offers an API that allows for the search and retrieval of the public, Bank documents available in the Documents & Reports site.  Records can be retrieved in a format useful for research and for inclusion in web sites outside of Documents & Reports and the World Bank. 
* **Limitations:** Imagination

In [8]:
#What follows is a simple request querying for records of documents related to wind turbines.  It also requests that the list returned be in a JSON format and
# that the country and document date be returned with each record.  (Other key, record fields, like display title and URL, are always returned):

#Bash

#curl -s "https://search.worldbank.org/api/v2/wds?format=json&qterm=wind%20turbine&fl=docdt,count,country"

#Python

!curl -s "https://search.worldbank.org/api/v2/wds?format=json&qterm=wind%20turbine&fl=docdt,count,country"




{"rows":10,"os":0,"page":1,"total":546,"documents":{"D11831032":{"id":"11831032","count":"Mexico","entityids":{"entityid":"000334955_20100222012800"},"docdt":"2006-04-24T00:00:00Z","abstracts":{"cdata!":"The objective of the Wind Umbrella (La\n            Venta II) Project aims to reduce greenhouse gases emissions\n            from power generation in Mexico and promote investment in\n            wind energy in Mexico to diversify the sources of power\n            generation in the country. There two components to the\n            projects. The first component of the project is purchase of\n            carbon emissions reductions. Certified emissions reduction\n            (CFE) uses a long-term power planning simulation model\n            (DECADES) to select future power plants that are needed for\n            the expansion of the country's electricity system. On\n            the basis of such modeling results it is possible to compare\n            alternative scenarios of new generat